In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../fip3")
from fip.profiles import CooccurrenceProfile
from fip import profiles
import re
from itertools import chain
from ast import literal_eval

In [3]:
#Loading zinc database from csv 
suppl_coco = pd.read_csv('../data/COCOECFP.csv', delimiter = ' ')
suppl_coco

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,CNP0000002,CC=C(N=CS)C(=O)OC1C(COC(C)=O)OC(C2(O)CC(=O)C(N...,660.183639,-2.08210,FJEMIESGEMWDOB-UHFFFAOYSA-N,FJEMIESGEMWDOB,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[39, 58, 80, 118, 140, 144, 147, 209, 229, 252..."
1,CNP0000003,COc1cc(C)cc2c1C(=O)OC1C(O)C=CC3c4c(cc(OC)c5c4C...,598.183897,3.63422,KLWKJVYCDFWQMK-UHFFFAOYSA-N,KLWKJVYCDFWQMK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 94, 249, 314, 467, 495, 497, 569, 587, 64..."
2,CNP0000004,COc1cc(C)cc2c1C(=O)OC1C(O)C=CC3c4c(cc(OC)c5c4C...,554.157682,3.32262,PTEKHLCNKCAXPH-UHFFFAOYSA-N,PTEKHLCNKCAXPH,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 249, 298, 314, 352, 467, 497, 569, 587, 6..."
3,CNP0000005,CC1(C)CC2C(OC(=O)c3ccccc3)C(OC(=O)c3ccccc3)CC(...,534.298139,6.87940,ZVAVQCZAGOKAMX-UHFFFAOYSA-N,ZVAVQCZAGOKAMX,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 70, 80, 114, 145, 147, 207, 270, 302, 341..."
4,CNP0000006,COc1cc(C)cc2c1C(=O)OC1C(O)C=CC3c4c(cc(O)c5c4C(...,540.142032,3.01962,UYIPOCQHTAYRMA-UHFFFAOYSA-N,UYIPOCQHTAYRMA,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 249, 298, 314, 352, 410, 457, 467, 497, 5..."
...,...,...,...,...,...,...,...,...,...
386292,CNP0436851,COc1cccc2ccc(C(C)=O)c(O)c12,216.078644,2.75660,XNWOWNYWQOTWIX-UHFFFAOYSA-N,XNWOWNYWQOTWIX,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[67, 202, 203, 230, 249, 650, 695, 725, 806, 8..."
386293,CNP0436852,COc1cc(OC)c2c(c1)C(OC)(c1ccc3c(c1O)C(=O)c1c(O)...,570.188983,4.77902,XWGVAZYMLDVIDS-UHFFFAOYSA-N,XWGVAZYMLDVIDS,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[5, 67, 84, 142, 148, 202, 206, 231, 249, 253,..."
386294,CNP0436853,CC1(C)C=Cc2cc(C3COc4c5c(cc(O)c4C3=O)OC(C)(C)C=...,420.157288,4.82510,XZXMEYSQXQNHCX-UHFFFAOYSA-N,XZXMEYSQXQNHCX,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 6, 53, 56, 84, 212, 276, 294, 314, 398, 43..."
386295,CNP0436854,CCCCCCCC1CC(=O)NC(CO)C(=O)NC(CO)C(=O)N2CCCC2C(...,1183.685261,-1.20930,YNLIJTVZSFUHKP-UHFFFAOYSA-N,YNLIJTVZSFUHKP,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 5, 19, 42, 80, 109, 115, 130, 222, 231, 23..."


In [4]:
suppl_coco= suppl_coco.sample(frac=1).reset_index(drop=True)

In [5]:
suppl_coco

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,CNP0239046,CCCCCC1C(=O)CCC1CC(=O)OC,226.156895,2.72510,KVWWIYGFBYDJQC-UHFFFAOYSA-N,KVWWIYGFBYDJQC,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 142, 195, 226, 294, 314, 334, 339, 370, 5..."
1,CNP0205086,Cc1c2c(cc3c1oc(=O)c1ccccc13)CN(c1cccc(Cl)c1)CO2,377.081871,5.26452,DWNKYEMDZRSBHK-UHFFFAOYSA-N,DWNKYEMDZRSBHK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[133, 137, 165, 216, 252, 314, 350, 382, 398, ..."
2,CNP0361840,COc1cc(C2CC(=O)Oc3ccc4c(=O)c(-c5ccccc5)coc4c32...,458.136553,4.92680,ZRAWZNDFONIPGO-UHFFFAOYSA-N,ZRAWZNDFONIPGO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[45, 60, 133, 145, 222, 227, 231, 249, 282, 28..."
3,CNP0264800,C=C1C(=O)OC2C=C(CO)CCC(OCC)C3=CC(OC3=O)C12,320.125988,1.05360,PTJPAEBQCZMNAV-UHFFFAOYSA-N,PTJPAEBQCZMNAV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[35, 80, 176, 222, 294, 314, 408, 514, 650, 65..."
4,CNP0396333,CC(=O)C(CC=C(C)C)C(C)=CCOc1ccc(CCNC(=O)c2ccccc...,419.246044,5.54570,FGXBFPFGWRVMGG-UHFFFAOYSA-N,FGXBFPFGWRVMGG,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 8, 80, 102, 197, 265, 310, 322, 367, 389, ..."
...,...,...,...,...,...,...,...,...,...
386292,CNP0315857,COC(=O)C1=COC(OC2OC(COC(=O)c3ccc(O)c(OC)c3)C(O...,554.163556,-0.63720,LIVSNGMPJUCRLO-UHFFFAOYSA-N,LIVSNGMPJUCRLO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[58, 80, 145, 147, 167, 231, 233, 249, 314, 42..."
386293,CNP0032553,Cc1c(CCC(=O)NCC(=O)NCC(=O)[O-])c(=O)oc2cc(O)c(...,395.065152,-0.62468,SSZVGMXOKPZNCV-UHFFFAOYSA-M,SSZVGMXOKPZNCV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 252, 254, 305, 314, 366, 406, 42..."
386294,CNP0223192,CC(=O)Oc1c(C)cc2c(c1O)C(=O)C(C1=CC(=O)c3cc(C)c...,490.089997,2.87624,JODPOMYHTFZNTK-UHFFFAOYSA-N,JODPOMYHTFZNTK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[16, 39, 84, 202, 224, 314, 333, 352, 516, 641..."
386295,CNP0356727,C[N+]([O-])=NCOC1OC(CO)C(O)C(OC2OC(COC3OC(CO)C...,576.201398,-6.99890,STHHNNUYQQUJGY-UHFFFAOYSA-N,STHHNNUYQQUJGY,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 80, 170, 222, 349, 443, 489, 588, 610, 65..."


In [7]:
suppl_coco.to_csv('../data/CP1/COCONUT/COCOrandom.csv', sep=' ', index = False)

In [13]:
suppl_coco = pd.read_csv('../data/CP1/COCONUT/COCOrandom.csv', delimiter = ' ')

In [16]:
df_split = np.array_split(suppl_coco, 10)

In [18]:
df_split[9]

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
347668,CNP0223145,C=C(C(=O)OC)C1CCC2(C)C(OC3OC(CO)C(O)C(O)C3O)CC...,460.230847,-1.30520,UIUPSHYSTLJURC-UHFFFAOYSA-N,UIUPSHYSTLJURC,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 209, 222, 224, 249, 485, 489, 610, 619, 6..."
347669,CNP0387243,COC(=O)C(O)Cc1ccccc1,180.078644,0.76300,NMPPJJIBQQCOOI-UHFFFAOYSA-N,NMPPJJIBQQCOOI,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 54, 79, 80, 147, 227, 389, 637, 650, 695, ..."
347670,CNP0059575,CS(=O)(=O)N1CCN2C(=O)c3cc(-c4ccco4)ccc3NC(=O)C2C1,375.088892,0.98470,ZEMLBLQINDSYAF-UHFFFAOYSA-N,ZEMLBLQINDSYAF,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[7, 74, 121, 142, 250, 268, 314, 350, 352, 474..."
347671,CNP0208831,CC1CCC2(C)C(C(=O)O)CCCC2C1(C)CCc1ccoc1,318.219495,5.15560,NTRSOYPAPOGZQP-UHFFFAOYSA-N,NTRSOYPAPOGZQP,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[5, 80, 120, 149, 158, 384, 389, 396, 407, 478..."
347672,CNP0364933,COc1ccc(C2=C(C=NCCc3c[nH]c4ccccc34)C(c3ccccc3C...,528.241293,7.74262,CCUNELBHYTXERR-UHFFFAOYSA-N,CCUNELBHYTXERR,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[14, 80, 94, 119, 140, 144, 249, 328, 335, 364..."
...,...,...,...,...,...,...,...,...,...
386292,CNP0315857,COC(=O)C1=COC(OC2OC(COC(=O)c3ccc(O)c(OC)c3)C(O...,554.163556,-0.63720,LIVSNGMPJUCRLO-UHFFFAOYSA-N,LIVSNGMPJUCRLO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[58, 80, 145, 147, 167, 231, 233, 249, 314, 42..."
386293,CNP0032553,Cc1c(CCC(=O)NCC(=O)NCC(=O)[O-])c(=O)oc2cc(O)c(...,395.065152,-0.62468,SSZVGMXOKPZNCV-UHFFFAOYSA-M,SSZVGMXOKPZNCV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 252, 254, 305, 314, 366, 406, 42..."
386294,CNP0223192,CC(=O)Oc1c(C)cc2c(c1O)C(=O)C(C1=CC(=O)c3cc(C)c...,490.089997,2.87624,JODPOMYHTFZNTK-UHFFFAOYSA-N,JODPOMYHTFZNTK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[16, 39, 84, 202, 224, 314, 333, 352, 516, 641..."
386295,CNP0356727,C[N+]([O-])=NCOC1OC(CO)C(O)C(OC2OC(COC3OC(CO)C...,576.201398,-6.99890,STHHNNUYQQUJGY-UHFFFAOYSA-N,STHHNNUYQQUJGY,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 80, 170, 222, 349, 443, 489, 588, 610, 65..."


In [19]:
p0 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp0_37902.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37902)
p0.df

value
feature1 feature2       
1028     1028       7952
         1039       1360
         1045        206
         105         220
         1057       6049
...                  ...
1341     801           1
1939     801           1
270      959           1
449      959           1
1125     503           1

[1671610 rows x 1 columns]

In [20]:
p6 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CP1/ZINC/ZINCp6_37901.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=37901)
p6.df

value
feature1 feature2       
1        1          9153
         1039       1554
         1057       8445
         1088       5205
         1144        193
...                  ...
1546     98            1
1743     1837          1
         777           1
         943           1
         98            1

[1673968 rows x 1 columns]

In [28]:
p6.add_another_cooccurrence_profile(p6)

In [29]:
p6.df

value
feature1 feature2       
1        1         18306
         1039       3108
         1057      16890
         1088      10410
         1144        386
...                  ...
1546     98            2
1743     1837          2
         777           2
         943           2
         98            2

[1673968 rows x 1 columns]